In [ ]:
import chromedriver_autoinstaller
from selenium import webdriver
import pandas as pd
import re
import math
import time
import os, sys
import requests
import numpy as np
import random
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.action_chains import ActionChains


In [ ]:
def open_chromedriver():
    chrome = chromedriver_autoinstaller.install(os.getcwd())

    options = webdriver.ChromeOptions()
    # options.headless = True

    driver = webdriver.Chrome(options=options)
    return(driver)


In [ ]:
def get_name():
    name_all = []
    for i in driver.find_elements_by_xpath('//*[@id="divL_AJAX"]/ul/li[*]/div/div/div[2]/span'):
        name_all.append(i.text)
        
    return(name_all)

In [ ]:
def get_complete():
    complete=[]
    for i in driver.find_elements_by_xpath('//*[@id="divL_AJAX"]/ul/li[*]/div/div/div[3]/div[1]/img[1]'):
        if i.get_attribute('alt') == '완강':
            complete.append('O')
        else :
            complete.append('X')
    return(complete)

In [ ]:
def get_title():
    title_all = []
    for i in driver.find_elements_by_xpath('//*[@id="divL_AJAX"]/ul/li[*]/div/div/div[3]/a[1]'):
        title_all.append(i.text)
        
    return(title_all)

In [ ]:
def get_link():
    link_all = []
    for i in driver.find_elements_by_xpath('//*[@id="divL_AJAX"]/ul/li[*]/div/div/div[3]/a[1]'):
        link_all.append(i.get_attribute('href'))
        
    return(link_all)

In [ ]:
def get_lec_id(link_all):
    lec_id_all=[]
    for i in link_all:
        lec_id_all.append(re.split('=',i)[1])
        
    return(lec_id_all)

In [ ]:
def get_data_2(link_all):
    data_2=[]
    num=0
    for link in link_all:
        
        time.sleep(random.random())
        
        num=num+1
        print('강의 구성 수집 중',num,'/',len(link_all))
        table=pd.read_html(link)

        for i in table:
            if list(i.columns)[0]!='강의명':
                continue
            else:
                table=i
                break

        if type(table)==list:
            data_2.extend(['강의 업로드 없음'])
            continue



        table=table.iloc[:,0:2]
        try:
            table = table[table.iloc[:,1].str.contains('[0-9]분')]

            list1 = list(range(1,len(table)+1))
            list2 = list(table.iloc[:,0])
            list3 = list(table.iloc[:,1])
            data_2.append([','.join(['(' + str(a) + ', ' + b+')' for a,b in zip(list1,list2)])])
        except:

            index_value= list(table.loc[pd.isnull(table).any(1), :].index.values)


            table = table.drop(index_value)

            table = table[table.iloc[:,1].str.contains('[0-9]분')]

            list1 = list(range(1,len(table)+1))
            list2 = list(table.iloc[:,0])
            list3 = list(table.iloc[:,1])
            data_2.append([','.join(['(' + str(a) + ', ' + b+')' for a,b in zip(list1,list2)])])

    return(data_2)

In [ ]:
def get_tch_id(link_all):
    tch_id =[]
    num=0
    for i in link_all:
        num=num+1
        print('teacher_id 수집 중',num,'/',len(link_all))
        res= requests.get(i)
        soup=bs(res.text, 'html.parser')
        tch_id.append(re.split('=',soup.find('span',{'class':'side'}).find('a')['href'])[1])
    return(tch_id)
        

In [ ]:
driver = open_chromedriver()

In [ ]:
driver.get('https://go3.etoos.com/lecture/mh_list.asp')

In [ ]:
name_all = []
complete = []
title_all=[]
link_all =[]
sub_all = []
num=0
len_xpath = len(driver.find_elements_by_xpath('//*[@id="divAJAX_AREA"]/div/ul[*]/li[*]/a'))
for i in range(1,3):
    for j in range(1,6):
        if i==2 and j==5:
            break
            
        driver.find_element_by_xpath('//*[@id="divAJAX_AREA"]/div/ul[%s]/li[%s]/a' %(i,j)).click()
        num=num+1
        print('이름, 완강 여부, 강의 제목, 강의 링크, 과목 수집 중',num,'/',len_xpath)
        if i==2 and j==2:
            for k in range(1,3):
                driver.find_element_by_xpath('//*[@id="divA_U"]/ul/li[%s]/a' %k).click()
            
                time.sleep(5)
                
            print(driver.find_element_by_xpath('//*[@id="divA_U"]/ul/li[%s]/a' %k).text)


            name= get_name()

            name_all.extend(name)
            link_all.extend(get_link())
            complete.extend(get_complete())
            title_all.extend(get_title())
            sub = driver.find_element_by_xpath('//*[@id="divA_U"]/ul/li[%s]/a' %k).text

            sub_all.extend([sub]*len(name))
            
            if k ==2 :
                driver.find_element_by_xpath('//*[@id="divAJAX_AREA"]/div/ul[%s]/li[%s]/a' %(i,j)).click()
                
                time.sleep(1.5)
            
        else:
            
            time.sleep(5)

            
            print(driver.find_element_by_xpath('//*[@id="divAJAX_AREA"]/div/ul[%s]/li[%s]/a' %(i,j)).text)

            name= get_name()

            name_all.extend(name)
            link_all.extend(get_link())
            complete.extend(get_complete())
            title_all.extend(get_title())
            sub = driver.find_element_by_xpath('//*[@id="divAJAX_AREA"]/div/ul[%s]/li[%s]/a' %(i,j)).text

            sub_all.extend([sub]*len(name))

In [ ]:
data_2 = get_data_2(link_all)

In [ ]:
tch_id=get_tch_id(link_all)

In [ ]:
lec_id = get_lec_id(link_all)

In [ ]:
data = pd.DataFrame({'사이트':['이투스']*len(sub_all),'과목':sub_all,'선생님':name_all,'선생님ID':tch_id,'강좌명':title_all,'강좌ID':lec_id,'완강여부':complete,'강의번호&챕터명&시간':data_2,'링크':link_all})

In [ ]:
writer = pd.ExcelWriter(r'이투스.xlsx', engine='xlsxwriter', options={'strings_to_urls': True})
data.to_excel(writer)
writer.close()